## 1) Tokenization

We usually look at grammar and meaning at the level of words, related to each other within sentences, within each document. So if we're starting with raw text, we first need to split the text into sentences, and those sentences into words -- which we call "tokens".

Now, you might imagine that the easiest way to identify sentences is to split the document at every period '.', and to split the sentences using white space to get the words.

In [ ]:
example_sents = example_text.split('.')
example_sents_toks = [sent.split(' ') for sent in example_sents]

for sent_toks in example_sents_toks:
    print(sent_toks)

This doesn't look right. Not all periods divide sentences (periods may also used in abbreviations), and not all sentences end in a period (some end in question marks or exclamation points). Words might be separated by not only single spaces, but also tabs or newlines. We can use the 're' package split method to use regular expressions that capture these various possibilities:

In [ ]:
import re

example_sents = re.split('(?<=[a-z])[.?!]\s', example_text)
example_sents_toks = [re.split('\s+', sent) for sent in example_sents]

for sent_toks in example_sents_toks:
    print(sent_toks)

This looks better, though we've lost the punctuation at the end of each sentence, except for the period at the end of the string (since we only split sentences on a period followed by white space). That last period has remained attached to the word 'out', since we only split words on white space. We could instead use 're.findall()' to search for all sequences of alphanumeric characters. This would split apart conjunctions, which might be useful if we want to consider 'I' and ''m' (short for 'am') to represent separate words.

We'll stop there, because NLTK provides handy classes to do this for us:

In [ ]:
import nltk

example_sents = nltk.sent_tokenize(example_text)
example_sents_toks = [nltk.word_tokenize(sent) for sent in example_sents]

for sent_toks in example_sents_toks:
    print(sent_toks)

## 2) Part-of-Speech Tagging

One of the fundamental aspects of words that we can use to begin to understand them is their parts of speech -- whether a word is a noun, verb, adjective, etc. Labeling each word in a sequence with its part of speech is called "part-of-speech tagging" or "POS tagging". A part of speech represents a syntactic function; the aim here is to identify the grammatical components of a sentence.

Some parts of speech are easier to spot than others, because they follow certain morphological patterns (e.g. verb endings). We can use regular expressions to find these recognizable words individually:

In [ ]:
patterns = [
    (r'.*ing$', 'VBG'),               # gerunds
    (r'.*ed$', 'VBD'),                # simple past
    (r'.*es$', 'VBZ'),                # 3rd singular present
    (r'.*ould$', 'MD'),               # modals
    (r'.*\'s$', 'NN$'),               # possessive nouns
    (r'.*s$', 'NNS'),                 # plural nouns
    (r'.*ly', 'RB'),                  # adverbs
    (r'^-?[0-9]+(.[0-9]+)?$', 'CD'),  # cardinal numbers
    (r'.*', 'NN')                     # nouns (default)
    ]

NLTK has a regular expression tagger that we can use, providing it with our own patterns. Let's see how many words we're able to tag in our example text.

In [ ]:
from nltk import RegexpTagger

regexp_tagger = RegexpTagger(patterns)

sent = nltk.word_tokenize("They refuse to permit us to obtain the refuse permit")
sent_tagged = regexp_tagger.tag(sent)

print(sent_tagged)

That didn't work so well, but no problem, this was a very naïve attempt. We can evaluate the accuracy nonetheless, using the POS-tagged Brown corpus:

In [ ]:
brown_tagged_sents = brown.tagged_sents(categories='news')
regexp_tagger.evaluate(brown_tagged_sents)

Some words may have different parts of speech depending on their context. In the sentence two fields above, the words "refuse" and "permit" both appear as verbs and as nouns. We can think of sequences that might indicate parts of speech. For instance, a word after "the" or "an" is more likely to be a noun, while a word after "did" or "does" is more likely to be a verb.

State-of-the-art POS taggers rely on probabilistic models and machine learning to tag tokens sequentially, or jointly across the whole sentence at the same time (finding the most likely combination of tags that make sense in relation to each other). Fortunately, you don't need to do this yourself. NLTK comes with an off-the-shelf POS tagger for English language text:

In [ ]:
sent = nltk.word_tokenize("They refuse to permit us to obtain the refuse permit")
sent_tagged = nltk.pos_tag(sent)
print(sent_tagged)

## 4) Phrase Chunking

We may want to work with larger segments of text than single words (but still smaller than a sentence). For instance, in the sentence "The black cat climbed over the tall fence", we might want to treat "The black cat" as one thing (the subject), "climbed over" as a distinct act, and "the tall fence" as another thing (the object). The first and third sequences are noun phrases, and the second is a verb phrase.

We can separate these phrases by "chunking" the sentence, i.e. splitting it into larger chunks than individual tokens. This is also an important step toward identifying entities, which are often represented by more than one word. You can probably imagine certain patterns that would define a noun phrase, using part of speech tags. For instance, a determiner (e.g. an article like "the") could be concatenated onto the noun that follows it. If there's an adjective between them, we can include that too.

To define rules about how to structure words based on their part of speech tags, we use a grammar (in this case, a "chunk grammar"). NLTK provides a RegexpParser that takes as input a grammar composed of regular expressions. The grammar is defined as a string, with one line for each rule we define. Each rule starts with the label we want to assign to the chunk (e.g. NP for "noun phrase"), followed by a colon, then an expression in regex-like notation that will be matched to tokens' POS tags.

We can define a single rule for a noun phrase like this. The rule allows 0 or 1 determiner, then 0 or more adjectives, and finally at least 1 noun. (By using 'NN.*' as the last POS tag, we can match 'NN', 'NNP' for a proper noun, or 'NNS' for a plural noun.) If a matching sequence of tokens is found, it will be labeled 'NP'.

In [ ]:
grammar = "NP: {<DT>?<JJ>*<NN.*>+}"

We create a chunk parser object by supplying this grammar, then use it to parse a sentence into chunks. The sentence we want to parse must already be POS-tagged, since our grammar uses those POS tags to identify chunks. Let's try this on the first sentence in the election-related sentences we just extracted.

In [ ]:
from nltk import RegexpParser

cp = RegexpParser(grammar)

sent_tagged = nltk.pos_tag(sent)
sent_chunked = cp.parse(sent_tagged)

print(sent_chunked)

When we called print() on this chunked sentence, it printed out a nested list of nodes. Some are phrases (labeled 'NP') and others that didn't get chunked into a phrase are just the original tagged tokens (e.g. the verb 'climbed').

The chunked sentence is actually an NLTK tree object, we can find out by calling type() on the output from the RegexpParser:

In [ ]:
type(sent_chunked)

The tree object has a number of methods we can use to interact with its components. For instance, we can use the method draw() to see a more graphical representation. This will open a separate window.

The tree is pretty flat, because we defined a grammar that only grouped words into non-overlapping noun phrases, with no additional hierarchy above them. This is sometimes referred to as "shallow parsing". We'll get to more complex parsing later.

In [ ]:
sent_chunked.draw()

If we want to move through the chunks and look at certain phrases, since the tree is essentially flat, we can use a 'for' loop to iterate through all of the nodes in the order they were printed above. Some of the nodes are themselves NLTK tree objects, containing the noun phrases we chunked. Other nodes are just tuples with a token and tag, that didn't make it into a chunk.

If a node is a tree object, it has a method label(), in this case marked 'NP'. It also has a method leaves() that will give us the list of tagged tokens (tuples) in the phrase. If we pull out the first token from each tuple, and concatenate these, we can get the original phrase back.

In [ ]:
for node in sent_chunked:
    if type(node)==nltk.tree.Tree and node.label()=='NP':
        phrase = [tok for (tok, tag) in node.leaves()]
        print(' '.join(phrase))

## 5) Named Entity Recognition

Once we have noun phrases separated out, we might find it useful to figure out what categories of things these nouns refer to. Especially if the noun phrase is a proper noun, i.e. a name of something, we might be able to tell if it is the name of a person, an organization, a place, or some other thing. Labeling noun phrases as different types of named entities is called "Named Entity Recognition" or "NER".

Named Entity Recognition involves meaning (semantics) as well as grammar (syntax). The name of a person or an organization might appear in the same place in the exact same sentence, so we also have to know something about existing person and organization names to be able to tell them apart. For that reason, NER taggers are usually trained from labeled training data, using supervised machine learning techniques. NLTK comes with a pre-trained NER tagger we can use for general English text:

In [ ]:
sent_nes = nltk.ne_chunk(sent_tagged)
print(sent_nes)

Now we can extract named entities and their NER categories. For instance, we might want to pull out a list of all of the organizations or people mentioned in the document:

In [ ]:
entities = {'ORGANIZATION':[], 'PERSON':[], 'LOCATION':[]}
for node in sent_nes:
    if type(node)==nltk.tree.Tree:
        phrase = [tok for (tok, tag) in node.leaves()]
        if node.label() in entities.keys():
            entities[node.label()].append(' '.join(phrase))

for key, value in entities.items():
    print(key, value)

## 6) Parsing

Breaking down parts of a sentence and identifying their grammatical roles constitutes parsing. There are two main types of parsing in NLP: constituency parsing and dependency parsing. 

### Constituency Parsing

The grammar rule we used to chunk noun phrases represents the first layer of a constituency parse. We can add rules to the grammar to identify other types of phrases as well.

The most common additional types of phrases are prepositional phrases and verb phrases. The standard approach is not just to put prepositions in prepositional phrases, and verbs into verb phrases. Instead, we label a verb plus the noun object it's acting on as a verb phrase. Similarly, a preposition plus the following noun is a prepositional phrase.

In other words, these phrases are nested. The noun phrases we identified with our first rule become components (or constituents) of the verb or prepositional phrases in the next layer up, until we get to the level of the sentence overall.

Let's add those additional phrase types to our grammar. We'll use three quotation marks to indicate a string that covers multiple lines.

In [ ]:
grammar = r"""
    NP: {<DT>?<JJ>*<NN.*>+}      # Chunk sequences of DT, JJ, NN
    PP: {<IN><NP>}               # Chunk prepositions followed by NP
    VP: {<VB.*><NP|PP|CLAUSE>+} # Chunk verbs and their arguments
    CLAUSE: {<NP><VP>}           # Chunk NP, VP into a clause
    """
cp = RegexpParser(grammar)

Now let's parse the first sentence in our election-related sentences again.

In [ ]:
sent_tagged = nltk.pos_tag(sent)
sent_chunked = cp.parse(sent_tagged)
print(sent_chunked)

We have more nested phrases now, NPs within PPs within VPs. We can draw the tree to better visualize the greater depth.

In [ ]:
sent_chunked.draw()

Since this is no longer a flat list of noun phrases or other nodes, it would be less wise to use a simple for loop to iterate through the nodes and look for certain phrases of interest. A tree structure is best traversed using recursion: define a function to perform the operations you want to do on one node, then have the function call itself on each of its children.

In [ ]:
def extract_nps_recurs(tree):
    nps = []
    if not type(tree)==nltk.tree.Tree:
        return nps
    if tree.label()=='NP':
        nps.append(' '.join([tok for (tok, tag) in tree.leaves()]))
    for subtree in tree:
        nps.extend(extract_nps_recurs(subtree))
    return nps

In [ ]:
extract_nps_recurs(sent_chunked)

### Dependency Parsing

Another way to parse sentences is to identify which words are syntactically dependent on other words, and what their dependency relationship is. Dependency parsing usually places the main verb of a sentence at the root of the tree, then assigns the verb's subject, direct object, and indirect objects as dependents. An indirect object will usually be connected to a root verb through a preposition. And nouns can have dependents too, which modify or are about some aspect of the noun.

> #### Prepositional phrase attachment
> 
> Dependency parsing is very complex; determining which words depend on which other words 
> involves not only part-of-speech tags, but other information that's more specific to each 
> verb or noun in the given sequence. Here's a classic example:
> 
> * "He ate pizza with olives."
> * "He ate pizza with a fork."
> 
> Which word in the sentence does the last word modify? In the first sentence, the olives are 
> on the pizza, they modify the noun. Saying "He ate with olives" wouldn't make sense without 
> the pizza. In the second sentence, we aren't talking about a thing called "a pizza with a 
> fork", that doesn't make sense. The fork modifies the verb "ate": "He ate with a fork".

Because of these nuances, dependency parsers are usually built using extensive training data, in the form of "treebanks" of sentences annotated with dependency relations. Several major dependency parsers are available in pre-trained form for English language text. It is also possible to train open-source dependency parsers on other publicly available treebanks (such as from the Universal Dependencies project, which offers annotated treebanks in many languages).

Today, we'll work with the Stanford Parser, which is part of the Stanford CoreNLP toolkit. Stanford CoreNLP provides a number of state-of-the-art NLP tools and is widely used by computer scientists as well as social scientists and humanists. It is written in Java, but there are APIs that enable you to access some of the tools from Python. Several of the most popular tools can be used through NLTK.

### Stanford CoreNLP

To get started, you'll need to have downloaded the Stanford Parser from this website: http://nlp.stanford.edu/software/lex-parser.html#Download and unzip it to a location on your computer that's easy to find (e.g. a folder called SourceCode in your Documents folder).

Then in Python, import the StanfordDependencyParser class from NLTK's parser package. You'll also need to import the module 'os' and set the following environment variables to the location on your computer where you put the unzipped Stanford Parser folder.

In [ ]:
import os
from nltk.parse.stanford import StanfordDependencyParser

os.environ['STANFORD_PARSER'] = '/PATH/stanford-parser-full-2016-10-31'
os.environ['STANFORD_MODELS'] = '/PATH/stanford-parser-full-2016-10-31'

Now let's create a dependency parser object and try parsing our election-related sentences.

In [ ]:
dependency_parser = StanfordDependencyParser(model_path="edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz")
sents_parsed = dependency_parser.parse_sents(sents)

The NLTK interface to the Stanford Parser returns an iterators (over iterators) over NLTK Dependency Graph objects. To be able to access the graph objects more than once, we can convert this into a list:

In [ ]:
sents_parseobjs = [obj for sent in sents_parsed for obj in sent]

In [ ]:
len(sents_parseobjs)

The graph object contains a method .tree() to depict the parse tree. (If we add .draw(), it will open in a separate window.)

In [ ]:
sents_parseobjs[0].tree()

This tree shows us the dependencies (i.e. the arcs), but it doesn't show us the labeled dependency relations, which are a huge part of the value of dependency parsing. In other words, it shows us that "investigation" and "evidence" are both dependents of the verb "produced", but it doesn't show which was the subject and which the object of the action.

The method .triples() extracts dependency triples of the form: ((head word, head tag), rel, (dep word, dep tag)). So for every head word - dependent word pair, it will give us a triple, with the dependency relation label in between. (The method .triples() also returns an iterator; here we'll just use a for loop to print out each triple.)

In [ ]:
for triple in sents_parseobjs[0].triples():
    print(triple)

This list of triples repeats a lot of head words, in order to capture all of their relations. Another format in which we can view the parse information is to convert it into CoNLL format. (CoNLL stands for the SIGNLL Conference on Computational Natural Language Learning; it organizes annual shared tasks relating to syntactic and semantic parsing.) The CoNLL formatted output is a string with one line for each word in the original sentence. The lines contain the word, its part-of-speech tag (two versions), the line number for the head word it is directly dependent on, and the label for that dependency relation.

In [ ]:
print(sents_parseobjs[0].to_conll(10))

In [ ]:
print(dir(sents_parseobjs[0]))